Goal : Figure out the canonicalization of the trained MPS

In [1]:
cd ..

/home/abhishekabhishek/git/UnsupGenModbyMPS


In [2]:
import numpy as np
from MPScumulant import MPS_c

In [27]:
m = MPS_c(16)
m.loadMPS('BS-MPS')

# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    #tn_core = np.swapaxes(m.matrices[i], 0, 1)
    tn_core = m.matrices[i]
    print(i, tn_core.shape)

0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 8)
3 (8, 2, 15)
4 (15, 2, 16)
5 (16, 2, 16)
6 (16, 2, 16)
7 (16, 2, 15)
8 (15, 2, 16)
9 (16, 2, 16)
10 (16, 2, 16)
11 (16, 2, 15)
12 (15, 2, 8)
13 (8, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)


Assuming the MPS is left-canonicalized - check the normalization conditions

In [28]:
mat_0 = m.matrices[0]
print(mat_0.shape)

(1, 2, 2)


Reshape to the left

In [29]:
mat_0 = mat_0.reshape(-1, mat_0.shape[2])
print(mat_0.shape)

(2, 2)


In [30]:
np.matmul(mat_0.conj().T, mat_0)

array([[1.0000000e+00, 2.1222909e-17],
       [2.1222909e-17, 1.0000000e+00]])

Check the canonicalization of the matrices in the trained MPS

In [37]:
def check_isometries(mps):
    for i in range(len(mps.matrices)):
        tn_core = mps.matrices[i]
        
        # convert the order-3 core tensor to a matrix
        core_mat = tn_core.reshape(-1, tn_core.shape[2])
        
        # check if the matrices are isometries or unitary
        left_isometry = np.allclose(
            np.eye(core_mat.shape[1]),
            np.matmul(core_mat.conj().T, core_mat)
        )
        
        right_isometry = np.allclose(
            np.eye(core_mat.shape[0]),
            np.matmul(core_mat, core_mat.conj().T)
        )
        
        print(i, tn_core.shape, left_isometry, right_isometry)

In [31]:
# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    tn_core = m.matrices[i]
    
    # convert the order-3 core tensor to a matrix
    core_mat = tn_core.reshape(-1, tn_core.shape[2])
    
    # check if the matrices are isometries or unitary
    left_isometry = np.allclose(
        np.eye(core_mat.shape[1]),
        np.matmul(core_mat.conj().T, core_mat)
    )
    
    right_isometry = np.allclose(
        np.eye(core_mat.shape[0]),
        np.matmul(core_mat, core_mat.conj().T)
    )
    
    print(i, tn_core.shape, left_isometry, right_isometry)

0 (1, 2, 2) True True
1 (2, 2, 4) True True
2 (4, 2, 8) True True
3 (8, 2, 15) True False
4 (15, 2, 16) True False
5 (16, 2, 16) True False
6 (16, 2, 16) True False
7 (16, 2, 15) True False
8 (15, 2, 16) True False
9 (16, 2, 16) True False
10 (16, 2, 16) True False
11 (16, 2, 15) True False
12 (15, 2, 8) True False
13 (8, 2, 4) True False
14 (4, 2, 2) False False
15 (2, 2, 1) False False


For some reason this is true upto tensor idx 13, and then this is no longer the case - **is this the case in general for left canonicalized MPS ?**

Also, check reshaping along the other axes:

In [18]:
# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    tn_core = m.matrices[i]
    
    # convert the order-3 core tensor to a matrix
    core_mat = tn_core.reshape(tn_core.shape[0], -1)
    
    # check if the matrices are isometries or unitary
    left_isometry = np.allclose(
        np.eye(core_mat.shape[1]),
        np.matmul(core_mat.conj().T, core_mat)
    )
    
    right_isometry = np.allclose(
        np.eye(core_mat.shape[0]),
        np.matmul(core_mat, core_mat.conj().T)
    )
    
    print(i, tn_core.shape, left_isometry, right_isometry)

0 (1, 2, 2) False False
1 (2, 2, 4) False False
2 (4, 2, 8) False False
3 (8, 2, 15) False False
4 (15, 2, 16) False False
5 (16, 2, 16) False False
6 (16, 2, 16) False False
7 (16, 2, 15) False False
8 (15, 2, 16) False False
9 (16, 2, 16) False False
10 (16, 2, 16) False False
11 (16, 2, 15) False False
12 (15, 2, 8) False False
13 (8, 2, 4) False False
14 (4, 2, 2) False False
15 (2, 2, 1) True True


ok, so this looks promising - we should be reshaping by the combining the first virtual axis with the physical axis

**Does calling `left_cano()` on this change anything?**

In [20]:
# create a copy of the MPS 
m_copy = MPS_c(16)
m_copy.matrices = m.matrices
m_copy.bond_dimension = m.bond_dimension

In [21]:
m_copy.left_cano()

bond: 0
bond: 1
bond: 2
bond: 3
bond: 4
bond: 5
bond: 6
bond: 7
bond: 8
bond: 9
bond: 10
bond: 11
bond: 12
bond: 13
bond: 14


In [22]:
# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    tn_core = m.matrices[i]
    
    # convert the order-3 core tensor to a matrix
    core_mat = tn_core.reshape(-1, tn_core.shape[2])
    
    # check if the matrices are isometries or unitary
    left_isometry = np.allclose(
        np.eye(core_mat.shape[1]),
        np.matmul(core_mat.conj().T, core_mat)
    )
    
    right_isometry = np.allclose(
        np.eye(core_mat.shape[0]),
        np.matmul(core_mat, core_mat.conj().T)
    )
    
    print(i, tn_core.shape, left_isometry, right_isometry)

0 (1, 2, 2) True True
1 (2, 2, 4) True True
2 (4, 2, 8) True True
3 (8, 2, 15) True False
4 (15, 2, 16) True False
5 (16, 2, 16) True False
6 (16, 2, 16) True False
7 (16, 2, 15) True False
8 (15, 2, 16) True False
9 (16, 2, 16) True False
10 (16, 2, 16) True False
11 (16, 2, 15) True False
12 (15, 2, 8) True False
13 (8, 2, 4) True False
14 (4, 2, 2) True False
15 (2, 2, 1) True False


Looks like that made all the core tensors in the MPS - left isometries - which is good news! - now we can try padding with 0 and repeating this process

In [32]:
# try to zero pad the TN core to make them powers of two
new_core_mats = m.matrices.copy()
new_bond_dims = m.bond_dimension.copy()
for i in range(1, len(m.matrices)-1):
    core_mat = m.matrices[i]
    pad_width_list = []
    for dim in range(len(core_mat.shape)):
        if core_mat.shape[dim]%2 == 0:
            pad_width_list.append((0, 0))
        else:
            pad_width_list.append((0, 1))
    core_mat = np.pad(core_mat, pad_width_list, mode='constant',
                           constant_values=0)
    new_core_mats[i] = core_mat
    new_bond_dims[i-1] = core_mat.shape[0]
    print(f"i = {i}, {core_mat.shape}, {core_mat.shape[0]}")

i = 1, (2, 2, 4), 2
i = 2, (4, 2, 8), 4
i = 3, (8, 2, 16), 8
i = 4, (16, 2, 16), 16
i = 5, (16, 2, 16), 16
i = 6, (16, 2, 16), 16
i = 7, (16, 2, 16), 16
i = 8, (16, 2, 16), 16
i = 9, (16, 2, 16), 16
i = 10, (16, 2, 16), 16
i = 11, (16, 2, 16), 16
i = 12, (16, 2, 8), 16
i = 13, (8, 2, 4), 8
i = 14, (4, 2, 2), 4


In [33]:
# create a new MPS with padded core tensors
m_pad = MPS_c(16)
m_pad.matrices = new_core_mats
m_pad.bond_dimension = new_bond_dims

In [34]:
# check the properties of the matrices in the MPS with padded core tensors
for i in range(len(m_pad.matrices)):
    tn_core = m_pad.matrices[i]
    print(i, tn_core.shape)
    
m_pad.bond_dimension

0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 8)
3 (8, 2, 16)
4 (16, 2, 16)
5 (16, 2, 16)
6 (16, 2, 16)
7 (16, 2, 16)
8 (16, 2, 16)
9 (16, 2, 16)
10 (16, 2, 16)
11 (16, 2, 16)
12 (16, 2, 8)
13 (8, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)


array([ 2,  4,  8, 16, 16, 16, 16, 16, 16, 16, 16, 16,  8,  4,  2,  1],
      dtype=int16)

In [36]:
# check the properties of the matrices in the MPS
for i in range(len(m_pad.matrices)):
    tn_core = m_pad.matrices[i]
    
    # convert the order-3 core tensor to a matrix
    core_mat = tn_core.reshape(-1, tn_core.shape[2])
    
    # check if the matrices are isometries or unitary
    left_isometry = np.allclose(
        np.eye(core_mat.shape[1]),
        np.matmul(core_mat.conj().T, core_mat)
    )
    
    right_isometry = np.allclose(
        np.eye(core_mat.shape[0]),
        np.matmul(core_mat, core_mat.conj().T)
    )
    
    print(i, tn_core.shape, left_isometry, right_isometry)

0 (1, 2, 2) True True
1 (2, 2, 4) True True
2 (4, 2, 8) True True
3 (8, 2, 16) False False
4 (16, 2, 16) True False
5 (16, 2, 16) True False
6 (16, 2, 16) True False
7 (16, 2, 16) False False
8 (16, 2, 16) True False
9 (16, 2, 16) True False
10 (16, 2, 16) True False
11 (16, 2, 16) False False
12 (16, 2, 8) True False
13 (8, 2, 4) True False
14 (4, 2, 2) False False
15 (2, 2, 1) False False


The matrices at idxs `[3, 7, 11]` are no longer left-isometries which they were previously. It's interesting to see that even after padding with zeros - the core tensors at `[4, 8, 12]` are still left isometries - **does this have something to do with the axis along which zeros are padded?**

Anyways, do a left canonicalization on the padded MPS and check if the resulting core tensors are left-isometries

In [39]:
m_pad.left_cano()

bond: 0
bond: 1
bond: 2
bond: 3
bond: 4
bond: 5
bond: 6
bond: 7
bond: 8
bond: 9
bond: 10
bond: 11
bond: 12
bond: 13
bond: 14
0 (1, 2, 2) True True
1 (2, 2, 4) True True
2 (4, 2, 8) True True
3 (8, 2, 16) True True
4 (16, 2, 16) True False
5 (16, 2, 16) True False
6 (16, 2, 16) True False
7 (16, 2, 16) True False
8 (16, 2, 16) True False
9 (16, 2, 16) True False
10 (16, 2, 16) True False
11 (16, 2, 16) True False
12 (16, 2, 8) True False
13 (8, 2, 4) True False
14 (4, 2, 2) True False
15 (2, 2, 1) True False


In [40]:
check_isometries(m_pad)

0 (1, 2, 2) True True
1 (2, 2, 4) True True
2 (4, 2, 8) True True
3 (8, 2, 16) True True
4 (16, 2, 16) True False
5 (16, 2, 16) True False
6 (16, 2, 16) True False
7 (16, 2, 16) True False
8 (16, 2, 16) True False
9 (16, 2, 16) True False
10 (16, 2, 16) True False
11 (16, 2, 16) True False
12 (16, 2, 8) True False
13 (8, 2, 4) True False
14 (4, 2, 2) True False
15 (2, 2, 1) True False


So, fingers crossed, as long as there are no bugs in this repo's code, the padded MPS is now left canonicalized and all the tensors are left isometries. **awesome!**

now, we need to figure out how to convert these left-isometries to unitaries and which of these matrices should we be even using ?

note that for the above MPS, the 4-qubit gate starts from site = 1 - just something to think about!

In [42]:
m_pad.bond_dimension, len(m_pad.bond_dimension)

(array([ 2,  4,  8, 16, 16, 16, 16, 16, 16, 16, 16, 16,  8,  4,  2,  1],
       dtype=int16),
 16)

In [51]:
# for each bond in the MPS, get me the size of the core tensors that it
# connects to
print('i, bond_dim, n_qubits, unitary_shape, left_shape, right_shape')
for i in range(len(m_pad.matrices)-1):
    bond_dim = m_pad.matrices[i].shape[2]
    n_qubits = int(np.log2(bond_dim)) + 1
    u_shape = (2**n_qubits, 2**n_qubits)
    
    left_shape, right_shape = m_pad.matrices[i].shape, m_pad.matrices[i+1].shape
    left_shape = (left_shape[0]*left_shape[1], left_shape[2])
    right_shape = (right_shape[0]*right_shape[1], right_shape[2])
    
    print(f'{i}, {bond_dim}, {n_qubits}, {u_shape}, {left_shape}, {right_shape}')

i, bond_dim, n_qubits, unitary_shape, left_shape, right_shape
0, 2, 2, (4, 4), (2, 2), (4, 4)
1, 4, 3, (8, 8), (4, 4), (8, 8)
2, 8, 4, (16, 16), (8, 8), (16, 16)
3, 16, 5, (32, 32), (16, 16), (32, 16)
4, 16, 5, (32, 32), (32, 16), (32, 16)
5, 16, 5, (32, 32), (32, 16), (32, 16)
6, 16, 5, (32, 32), (32, 16), (32, 16)
7, 16, 5, (32, 32), (32, 16), (32, 16)
8, 16, 5, (32, 32), (32, 16), (32, 16)
9, 16, 5, (32, 32), (32, 16), (32, 16)
10, 16, 5, (32, 32), (32, 16), (32, 16)
11, 16, 5, (32, 32), (32, 16), (32, 8)
12, 8, 4, (16, 16), (32, 8), (16, 4)
13, 4, 3, (8, 8), (16, 4), (8, 2)
14, 2, 2, (4, 4), (8, 2), (4, 1)


From a first scan, it looks like we should be using the right handside tensor of each bond to build out unitaries using the technique described in the paper of extending rows or columns (**it's always columns in our case**)

also the great news is the first 3-tensors we need are already unitaries i.e. corresponding to bonds 0, 1, and 2 are already unitaries which supports the idea above.

for the second stage i.e. mapping rectangular isometries to unitaries - we can just work with one of the above tensors e.g. for bond 3 = `(32, 16)` and then simply apply the approach to the rest of them.

In [56]:
core_tensor_15 = m_pad.matrices[15]
print(core_tensor_15.shape)

(2, 2, 1)


In [57]:
core_mat_15 = core_tensor_15.reshape(-1, core_tensor_15.shape[2])
print(core_mat_15.shape)

(4, 1)


In [58]:
# check the isometry conditions
left_isometry = np.allclose(
    np.eye(core_mat_15.shape[1]),
    np.matmul(core_mat_15.conj().T, core_mat_15)
)

right_isometry = np.allclose(
    np.eye(core_mat_15.shape[0]),
    np.matmul(core_mat_15, core_mat_15.conj().T)
)

print(left_isometry, right_isometry)

True False


In [59]:
core_mat_15

array([[ 0.50002703],
       [ 0.49997297],
       [-0.49997296],
       [ 0.50002703]])

In [65]:
isometry_to_unitary(core_mat_15)

AssertionError: 

In [67]:
core_mat_15.shape

(4, 1)

In [68]:
import scipy

In [72]:
x = scipy.linalg.null_space(core_mat_15.conj().T)

In [73]:
x.shape

(4, 3)

In [75]:
np.matmul(core_mat_15.conj().T, x)

array([[-3.38643747e-17,  7.19768631e-17, -6.36601364e-17]])

These are very close to zero which is what we want

In [77]:
np.matmul(x.conj().T, x)

array([[1.00000000e+00, 1.27873476e-18, 0.00000000e+00],
       [1.27873476e-18, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [78]:
np.allclose(
    np.eye(x.shape[1]),
    np.matmul(x.conj().T, x)
)

True

This basically gives us what we need to construct the unitary

In [80]:
u = np.hstack((core_mat_15, x))

In [81]:
# check if the matrices are isometries or unitary
left_isometry = np.allclose(
    np.eye(u.shape[1]),
    np.matmul(u.conj().T, u)
)

right_isometry = np.allclose(
    np.eye(u.shape[0]),
    np.matmul(u, u.conj().T)
)

In [82]:
left_isometry, right_isometry

(True, True)

I think it's time to write some scripts to peform this process